In [1]:
import os
import torch
import pandas as pd
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, random_split, DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import datetime
from tensorflow import summary
from collections import OrderedDict
%matplotlib inline

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [3]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 689F-4E1C

 Directory of C:\Users\vidas\OneDrive\University\Data Science Program

06/15/2022  03:45 PM    <DIR>          .
06/02/2022  07:55 PM    <DIR>          ..
06/15/2022  02:57 PM    <DIR>          .ipynb_checkpoints
06/15/2022  02:52 PM    <DIR>          Dog Breed Classification
06/15/2022  03:45 PM            14,503 Dog-Breed-Classification.ipynb
06/13/2022  04:11 PM            95,764 Lab7_PyTorch_CNNs.ipynb
06/13/2022  05:26 PM        10,696,299 Lab7_With_Tensorboard.ipynb
06/13/2022  05:26 PM        10,705,664 Lab8_Transfer_Learning.ipynb
06/15/2022  03:39 PM    <DIR>          logs
06/02/2022  07:55 PM    <DIR>          Machine-Learning-Class
06/14/2022  04:18 PM             3,957 pytorch_finetune_tensorboard.py
06/15/2022  02:47 PM    <DIR>          stanford-dogs-dataset
               5 File(s)     21,516,187 bytes
               7 Dir(s)  433,713,774,592 bytes free


In [4]:
dataset = ImageFolder('./stanford-dogs-dataset/images/Images')

In [5]:
len(dataset) #check how many images

20580

In [6]:
len(dataset.classes) #check how many dog breeds

120

In [23]:
dataset.classes[:3]

['n02085620-Chihuahua', 'n02085782-Japanese_spaniel', 'n02085936-Maltese_dog']

In [8]:
#Formatting
#taking the 120 different total classes and renaming them into a nicer format.

dog_breeds = []

def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

for n in dataset.classes:
    dog_breeds.append(rename(n))
    
dog_breeds[:3]

['Chihuahua', 'Japanese spaniel', 'Maltese dog']

In [9]:
# set a random seed so the data is reproducible in the same way
random_seed = 42
torch.manual_seed(random_seed);

In [10]:
#Split the dataset into Training, Validation, and Testing

test_pct = 0.3 #test size will be 30% of dataset
test_size = int(len(dataset)*test_pct)
dataset_size = len(dataset) - test_size

val_pct = 0.1 #validation set will be 10% of dataset
val_size = int(dataset_size*val_pct)
train_size = dataset_size - val_size #training set will be 90% of dataset


train_size, val_size, test_size

(12966, 1440, 6174)

In [11]:
#split the training dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
len(train_dataset), len(val_dataset), len(test_dataset)

(12966, 1440, 6174)

In [12]:
class DogBreedDataset(Dataset):
    
    def __init__(self, ds, transform=None):
        self.ds = ds
        self.transform = transform
        
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        img, label = self.ds[idx]
        if self.transform:
            img = self.transform(img)  
            return img, label

In [13]:
#Transforms

imagenet_stats = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)


train_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

val_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

In [ ]:

def imshow(img):
  img = img / 2 + 0.5
  np_img = img.numpy()
  plt.imshow(np.transpose(np_img, (1, 2, 0)))
  plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))

In [14]:
train_dataset = DogBreedDataset(train_dataset, train_transform)
val_dataset = DogBreedDataset(val_dataset, val_transform)
test_dataset = DogBreedDataset(test_dataset, test_transform)

In [15]:
batch_size = 16

# Create DataLoaders
train_dl = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_dl = DataLoader(val_dataset, batch_size*2, num_workers=2, pin_memory=True)
test_dl = DataLoader(test_dataset, batch_size*2, num_workers=2, pin_memory=True)

In [ ]:

def imshow(img):
  img = img / 2 + 0.5
  np_img = img.numpy()
  plt.imshow(np.transpose(np_img, (1, 2, 0)))
  plt.show()

dataiter = iter(train_dl)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))

In [16]:
class Net(nn.Module):

  def __init__(self):
    super().__init__()
    k_size = 5
    n_input_channels = 3 # rgb
    n_output_channels_1 = 6
    self.conv1 = nn.Conv2d(n_input_channels, n_output_channels_1, k_size)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(n_output_channels_1, 16, k_size)
    self.fc1 = nn.Linear(16 * k_size * k_size, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    out = self.conv1(x)
    out = F.relu(out)
    out = self.pool(out)

    out = self.conv2(out)
    out = F.relu(out)
    out = self.pool(out)

    out = torch.flatten(out, 1)

    out = self.fc1(out)
    out = F.relu(out)

    out = self.fc2(out)
    out = F.relu(out)

    out = self.fc3(out)
    #out = F.softmax(out, dim=1)
    return out

net = Net()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [18]:
current_time = str(datetime.datetime.now().timestamp())
train_log_dir = 'logs/tensorboard/train/' + current_time
val_log_dir = 'logs/tensorboard/val/' + current_time
train_summary_writer = summary.create_file_writer(train_log_dir)
val_summary_writer = summary.create_file_writer(val_log_dir)

In [19]:
def train():
  for epoch in range(30):

    correct = 0
    total = 0
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
      inputs, labels = data
      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()
      outputs = net(inputs)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

      loss = F.cross_entropy(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
             
    with train_summary_writer.as_default():
      summary.scalar('train loss', running_loss / i, step=epoch)
      summary.scalar('train accuracy', correct / total, step=epoch)

    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad():
      for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss = F.cross_entropy(outputs, labels)
        running_loss += loss.item()

    with val_summary_writer.as_default():
      summary.scalar('validation loss', running_loss / i, step=epoch)
      summary.scalar('validation accuracy', correct / total, step=epoch)

In [20]:
%tensorboard --logdir ./logs/tensorboard

Reusing TensorBoard on port 6006 (pid 2088), started 0:01:15 ago. (Use '!kill 2088' to kill it.)